In [1]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
import sys 
import matplotlib 
import os
%matplotlib inline
import json

with open('data.json', 'r') as f:
    df = json.load(f)
print ('main data fields: {}'.format(df.keys()))
print ('number of tables to be clustered: {}'.format(len(df['tables'])))

main data fields: dict_keys(['lineage', 'tables', 'joins', 'columns', 'tags'])
number of tables to be clustered: 2219


In [2]:
# df is the master dataframe read direction from jason, and generate text fields for each table, 
# including a column that collected all the text information for each data table
def generate_table_text_df(df):
    df_tables = pd.DataFrame(df['tables'])
    df_columns = pd.DataFrame(df['columns'])
    tables = df_tables[['fp', 'id', 'datasource_name', 'name', 'schema_name', 'title', 'description', 'popularity']]
    # extract columns from original table information
    columns = df_columns[['table_fp', 'table_id', 'name', 'table_name', 'title', 'popularity', 'fp']]
    # remove the two column without names
    columns = columns.iloc[2:]
    # get column names for each table
    col_names = columns.groupby('table_fp')['name'].apply(' '.join).reset_index()
    
    table_col_names = tables.join(col_names.set_index('table_fp'), on = 'fp', rsuffix = '_col')
    table_text = table_col_names[['fp', 'id', 'datasource_name', 'name', 'schema_name', 'title', 'description', 'name_col']].copy()
    table_text[['fp', 'id','datasource_name', 'name', 'schema_name', 'title', 'description', 'name_col']] = table_text[
        ['fp', 'id','datasource_name', 'name', 'schema_name', 'title', 'description', 'name_col']].astype('str')
    table_text['text_info'] = table_text[['datasource_name', 'name', 
                         'schema_name', 'title', 'name_col']].apply(lambda x:  " ".join(x), axis=1)
    return table_text, col_names, columns

In [3]:
table_text, col_names, columns  = generate_table_text_df(df)
columns.head()
#table_text.head()['text_info']

,table_fp,table_id,name,table_name,title,popularity,fp
2,3367216718368952925,3,table1_pk_col,table1,,0.0,2165169567917872817
3,3367216718368952925,3,table1_char_col,table1,,0.0,703205646241500466
4,3367216718368952925,3,table1_varchar_col,table1,,0.0,2376408241062256224
5,3367216718368952925,3,table1_date_col,table1,,0.0,1267002101680272448
6,5955604996812906401,4,fips,census_data1,,0.0,1148298353030312426


In [4]:
def generate_table_tags_df(df):
    num_table_tags = 0
    num_column_tags = 0
    table_id = []
    column_id = []
    tags = []
    for key, value in df['tags'].items():    
        for i in range(len(value)):
            if value[i][0] == int(27):
                num_table_tags += 1
                table_id.append(value[i][1])
                tags.append(key)
            elif value[i][0] == int(1):
                num_column_tags += 1
                column_id.append(value[i][1])
    print ("{} out of 2219 tables have tags, and {} out of 30K+ columns have tags".
           format(num_table_tags, num_column_tags))
    print ('there are {} unique tags for tables'.format(len(set(table_id))))
    return pd.DataFrame({'table_id' : table_id, 'tags' : tags})

In [5]:
table_tags_df = generate_table_tags_df(df)
table_tags_df = table_tags_df.groupby('table_id')['tags'].apply(' '.join).reset_index()
table_tags_df.tail()

87 out of 2219 tables have tags, and 39 out of 30K+ columns have tags
there are 48 unique tags for tables


,table_id,tags
43,1285,PII
44,1286,transactions trade
45,1290,excel
46,1730,PII Healthcare
47,2110,Sales PII


In [6]:
table_text.to_csv('table_df_text.csv', index = False)
table_tags_df.to_csv('table_w_tags.csv', index = False)